In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

def maskSR(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

# 移除山地阴影
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)# .reproject('EPSG:3857',None,30)
def removeShadow(image):
    azimuth = image.get('SOLAR_AZIMUTH_ANGLE')
    zenith = image.get('SOLAR_ZENITH_ANGLE')
    return image.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True))

# 计算研究区域内，影像面积
def ImageArea(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.set({'imagearea': imageareas.get('B2')})

# 数据集

In [5]:
lt2_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31')\
    .map(water_index)
print(lt2_images.size().getInfo())
Map.addLayer(lt2_images.map(maskL8sr).mosaic(),visParams,'landsat image')
Map.addLayer(lt2_images.first(),visParams,'first image')

134


In [6]:
def validPercentage(image):
    mask_area = maskL8sr(image).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageArea = area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 30,
        'maxPixels': 1e14,
    })
    mask_imageArea = mask_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 30,
        'maxPixels': 1e14,
    })
    per = ee.Number(100).subtract(ee.Number(mask_imageArea.get('B2')).divide(ee.Number(imageArea.get('B2'))).multiply(100))
    return image.set({'validPixelPer': per})

In [8]:
q = lt2_images.map(validPercentage).filter(ee.Filter.lt('validPixelPer',95)).filter(ee.Filter.gte('validPixelPer',5))
print(q.size().getInfo())
print(q.aggregate_array('validPixelPer').getInfo())

112
[15.159611635906316, 15.047348171749178, 71.63437517571958, 18.224586829569134, 19.549881758258607, 20.847628177409945, 21.59255993542854, 44.88356372876602, 17.4121952416709, 74.15732915760505, 30.33337499667026, 92.91274218430263, 41.15103293298954, 66.78066002449785, 94.40584166777003, 63.60369546043121, 48.058519923105166, 73.30074995272173, 74.33628924144327, 27.373669323378394, 21.953384855484487, 64.7515647418269, 26.078664279049704, 11.780366299688694, 39.59024835990434, 33.62541247204787, 70.66393002816025, 31.3726515043342, 71.97041568447867, 63.297319569489325, 44.30043186050432, 54.481337174209, 60.95568964025778, 92.85953654239688, 89.3643055288972, 72.54326582910943, 91.23613627982688, 80.60212889954623, 94.35011792710773, 75.87558425078116, 72.90890098917683, 18.388091668142536, 93.82069556316432, 66.25851809839374, 53.85466148276679, 21.390628548426093, 31.65278242721989, 68.14104479434447, 27.827120025861817, 70.89799079761029, 79.32604064550611, 84.36395364652631,

# RF训练

In [9]:
## 导入采集的样本集
points_collection = ee.FeatureCollection('users/311605001111/qinghai_2018')
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))
a = points_collection.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())


# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# 分类标签
label = 'waterclass'
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [90.6397502534006, 35.140012738196056]}, 'id': '000000000000000003c3', 'properties': {'AWEI': -5345, 'B2': 1231, 'B3': 1824, 'B4': 2401, 'B5': 2985, 'B6': 3863, 'B7': 3456, 'NDVI': 0.10842926055, 'mNDWI': -0.3585370183, 'ndvi_mndwi': 0.4669662714, 'waterclass': 0}}
total sample number:2199
水体样本的数目： 86


# 筛选出需要（不需要）校正的图像

In [ ]:
#对影像进行分类,计算水体面积
def waterArea(image):
    classified_image = maskL8sr(image).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    water_area = classified_image.multiply(ee.Image.pixelArea()).divide(1e6)
    waterarea = water_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })
    return image.set({'waterarea': waterarea.get('waterclass')})  

# 需要校正的图像
corrected_image = lt2_images.map(waterArea).filter(ee.Filter.gt('waterarea',10))
print(corrected_image.aggregate_array('waterarea').getInfo())
print(corrected_image.size().getInfo())
# 不需要校正的图像
No_corrected_image = lt2_images.map(waterArea).filter(ee.Filter.lte('waterarea',10))
print(No_corrected_image.aggregate_array('waterarea').getInfo())
print(No_corrected_image.size().getInfo())

# 图像校正函数

In [13]:
def AutomaticCorrection(img):
    image = maskL8sr(img).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    basemap = ee.Image.constant(0).toFloat().clip(img.geometry()).rename('waterclass')
    class_image = ee.ImageCollection([basemap,image.remap([0,1],[1,2]).rename('waterclass')]).sum()
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': img.geometry(),
        'scale': 30,
        'bestEffort': True
    })
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water.set({'occurrence_threshold':occurrence_threshold})

In [14]:
a = q.map(AutomaticCorrection)
print(a.aggregate_array('occurrence_threshold').getInfo())

EEException: Collection.reduceColumns: Error in map(ID=LC08_140036_20180125):
Dictionary.get: Dictionary does not contain key: bucketMeans.

# 计算水体频率

In [ ]:
## 水体频率
# 校正图像
corrected_waterPixel = corrected_image.map(AutomaticCorrection).sum()
corrected_validPixel = corrected_image.count().select('B3').rename('count')
# 未校正图像
noCorrected_waterPixel = No_corrected_image.map(lambda i : maskL8sr(i).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')).sum()
noCorrected_validPixel = No_corrected_image.map(lambda i : maskL8sr(i)).count().select('B3').rename('count')
# 水体频率
waterPixel_image = ee.ImageCollection([corrected_waterPixel,noCorrected_waterPixel]).sum()
validPixel_image = ee.ImageCollection([corrected_validPixel,noCorrected_validPixel]).sum()
waterfrequency = waterPixel_image.select('waterclass').divide(validPixel_image.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")
permanentwater = waterfrequency.gte(0.75)
Map.addLayer(permanentwater.selfMask(),{'palette':['blue']},"permanent water")

# 制图

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
%pylab inline
region = [89.8565, 34.6503,91.4664, 35.8282] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'waterbody frequency(Auto-Correction/validPixel/2018)', fontsize=20)

In [ ]:
# permanent waterbody
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8565, 34.6503,91.4664, 35.8282] 
vis = {'palette':['D3D3D3','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'permanent water(Auto-Correction/validPixel/2018)', fontsize=20)